<a href="https://colab.research.google.com/github/binh1206/PINN-SciANN/blob/main/DampedHarmonicOscillator/Inverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.10 keras==2.10 sciann
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install ()
!conda config --add channels conda-forge
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [ ]:
import numpy as np
import matplotlib . pyplot as plt
import sciann as sn
from sciann.utils.math import diff, sign, sin

In [ ]:
w0 = 20
sigma = 2 # under-damped sigma < w0
w = np.sqrt(w0**2 - sigma**2)
phi = np.arctan(-(0+sigma)/w)
A = 1/(2*np.cos(phi))
def func_x(t):
  return np.exp(-sigma*t)*2*A*np.cos(w*t+phi) # nghiem chuan
# khoi tao bo du lieu:
t_data = np.linspace(0,0.8,100) # 100 datapoints
x_data = func_x(t_data)

In [ ]:
t = sn.Variable('t') # dau vao
x = sn.Field('x') # dau ra
x = sn.Functional(
    fields=[x],
    variables=[t],
    hidden_layers=8*[20], # mang than kinh 8*20
    activation='sin') # ham kich hoat

In [ ]:
np.random.seed(20)
# sigma can tim:
sigma_ = sn.Parameter(np.random.rand(), inputs=[t], name='sigma_')
print(f'initial sigma_ = {sigma_.value}')
x_t = diff(x, t)
x_tt = diff(x, t, order=2)
ode = (x_tt + 2*sigma*x_t + (20**2)*x) # ODE

In [ ]:
m = sn.SciModel([t], [x, ode], loss_func='mse', optimizer='adam')
h = m.train(
    [t_data],
    [x_data, 'zeros'],
    epochs=10000,
    adaptive_weights=100,
    learning_rate=1e-2,
    batch_size=50,
    verbose=1)
# ve do thi Loss vs Epoch
plt.semilogy(h.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
print('sigma is identified by SciANN = {}'.format(sigma_.value))
percent_err = np.abs(sigma_.value - sigma)/(sigma_.value)*100
print(f'error(%) = {percent_err}')